In [1]:
import numpy as np
import pandas as pd

In [2]:
longtime_feat = pd.read_csv('feat_input/longtime_feat.csv')
encoded_feat = pd.read_csv('feat_input/feat_stats_encoding.csv')

In [3]:
feat_basic_csv = pd.read_csv('feat_input/basic_feat.csv')

In [4]:
feat_csv = feat_basic_csv.merge(longtime_feat, on='listing_id')
feat_csv = feat_csv.merge(encoded_feat, on='listing_id')

In [5]:
feat_csv.head()

,bathrooms,bedrooms,building_id,display_address,features,interest_level,latitude,listing_id,longitude,manager_id,...,price_std_bathrooms,price_median_bathrooms,price_max_bathrooms,price_min_bathrooms,manager_id_low,manager_id_medium,manager_id_high,price_manager_id_mean_pred_0,price_manager_id_mean_pred_1,price_manager_id_mean_pred_2
0,2.0,3,f7fc4fd7b6b80615ebfce3e212e17cce,Hausman Street,"[Pre-War, Dishwasher, Hardwood Floors]",1,40.7245,6818139,-73.9389,198d2e96429920ff71cd06ddff323713,...,1983.841877,5200.0,50550,800,0.727554,0.179567,0.092879,3088.333333,0.000000,0.000000
1,1.0,0,45c480f3a0771a4e6ddf7a81790a0b07,E 78 Street,"[Loft, Laundry in Unit, Hardwood Floors, Dogs ...",1,40.7706,7088908,-73.9501,136b844f49c4eae017d6cabe8ca03b9d,...,17543.181907,2900.0,4490000,1,0.745455,0.678175,0.737513,2655.636364,468.381818,153.454545
2,1.0,0,b0d419ddd5ff150f96b328c61ee6e7e2,W 83rd St,"[Furnished, Short Term Allowed]",2,40.7855,7122047,-73.9752,0a4de95e82c122b78494dcea6cb9f44d,...,17543.181907,2900.0,4490000,1,0.884674,0.294118,0.488038,985.294118,727.294118,810.000000
3,1.0,3,c3d5bcc96eb5d960b9a3feb67e71115b,40th St,[],0,40.6444,7169671,-73.9936,324631e1aacaa41bc4475f879756bbb6,...,17543.181907,2900.0,4490000,1,0.963549,0.080645,0.656988,2185.177419,141.935484,51.532258
4,1.0,3,3989b2e8d5eb6cd4ccfc8fa4257b277b,W 144th Beautiful Spacious Renovated Secure L...,"[Cats Allowed, Dogs Allowed, No Fee, Reduced F...",1,40.8260,7168565,-73.9519,2a4e0d2668fde3afa267d312b3a2a9d8,...,17543.181907,2900.0,4490000,1,0.264957,0.683939,0.966049,581.196581,1075.854701,445.299145


In [5]:
#feat_csv = feat_csv.sort_values('listing_id', ascending=True).reset_index()

In [6]:
import xgboost as xgb
def xgb_cv(dtrain, num_rounds = 50000, early_stop_rounds=250):
    print('Start xgboost cross-validation')
    params = {'booster': 'gbtree',
              'objective': 'multi:softprob',
              'eval_metric': 'mlogloss',
              'gamma': 1,
              'min_child_weight': 1.5,
              'max_depth': 5,
              'lambda': 10,
              'subsample': 0.7,
              'colsample_bytree': 0.7,
              'colsample_bylevel': 0.7,
              'eta': 0.03,
              'tree_method': 'exact',
              'seed': 36683,
              'nthread': 4,
              'num_class': 3,
              'silent': 1
              }
    xgb2cv = xgb.cv(params=params,
                    dtrain=dtrain,
                    num_boost_round=num_rounds,
                    nfold=5,
                    stratified=True,
                    verbose_eval=50,
                    early_stopping_rounds=early_stop_rounds)
    return xgb2cv

In [7]:
from sklearn.preprocessing import LabelEncoder
categorical = ["display_address", "manager_id", "building_id", "street_address"]
for f in categorical:
    if feat_csv[f].dtype == 'object':
        lbl = LabelEncoder()
        lbl.fit(list(feat_csv[f].values))
        feat_csv[f] = lbl.transform(list(feat_csv[f].values))

In [8]:
feat_csv.drop(['features'], axis=1, inplace=True)

In [9]:
train_csv = feat_csv[feat_csv.interest_level != -1].copy()
test_csv = feat_csv[feat_csv.interest_level == -1].copy()
y_train_csv=train_csv["interest_level"]
x_train_csv = train_csv.drop(['interest_level'], axis=1)
x_train_csv.columns.values

array(['bathrooms', 'bedrooms', 'building_id', 'display_address',
       'latitude', 'listing_id', 'longitude', 'manager_id', 'price',
       'street_address', 'feature_num', 'photo_num', 'desc_wordcount',
       'distance_city', 'day_of_year', 'created_month', 'created_day',
       'created_hour', 'day_of_week', 'price_bed_rt', 'price_bath_rt',
       'price_room_rt', 'bed_bath_rt', 'bed_bath_dif', 'bed_bath_sum',
       'bed_room_rt', 'time_stamp', 'img_sizes_mean',
       'jwd_type_low_than_num', 'jwd_type_all', 'jwd_type_rt',
       'building_zero_num', 'bathrooms_size_manager_id',
       'bathrooms_mean_manager_id', 'bathrooms_std_manager_id',
       'bathrooms_median_manager_id', 'bathrooms_max_manager_id',
       'bathrooms_min_manager_id', 'bedrooms_size_manager_id',
       'bedrooms_mean_manager_id', 'bedrooms_std_manager_id',
       'bedrooms_median_manager_id', 'bedrooms_max_manager_id',
       'bedrooms_min_manager_id', 'latitude_size_manager_id',
       'latitude_mean_mana

In [ ]:
#dtrain = xgb.DMatrix(x_train_csv, label=y_train_csv)
#_ = xgb_cv(dtrain)

Start xgboost cross-validation
[0]	train-mlogloss:1.0785+3.18584e-05	test-mlogloss:1.07876+6.53024e-05
[50]	train-mlogloss:0.660557+0.000810693	test-mlogloss:0.670132+0.0014998
[100]	train-mlogloss:0.571757+0.000768113	test-mlogloss:0.588446+0.00239705
[150]	train-mlogloss:0.538888+0.000720602	test-mlogloss:0.56199+0.00230571
[200]	train-mlogloss:0.519995+0.000498575	test-mlogloss:0.549604+0.00220778
